In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [7]:
def eval_net(net, dataset, gpu=False):
    """Evaluation with the dice coefficient"""
    net.eval()
    tot = 0
    for i, b in enumerate(dataset):
        traj = b[0]
        true_label = b[1]

        #traj = torch.from_numpy(traj).unsqueeze(0)
        #true_label = torch.from_numpy(true_label).unsqueeze(0)

        if gpu:
            traj = traj.cuda()
            true_label = true_label.cuda()

        pred = net(traj.double())[0]

        if (torch.argmax(pred) - true_label == 0):
            tot += 1
    return tot / (i + 1)

def one_hot(x):
    """One hot encoding the label"""
    encoded = np.zeros(3)
    encoded[int(x)] = 1.
    return encoded

def normalize(x):
    """Maps the coords to range [-1,1]"""
    return x / 200

In [8]:
#Training conv_model
import torch
from models import conv_model, lin_model
from data_utils import TrajDataset, FrameDataset, get_ids, split_train_val

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# Parameters
params = {'batch_size': 1,
          'shuffle': True,
          'num_workers': 4}
epochs = 20
lr = 0.001              #learning rate
save_cp = 1           #save checkpoints or not
net = conv_model(n_channels=801, n_classes=3).double()      #model to use
batch_size = params['batch_size']
dir_checkpoint = './checkpoints/'
val_scores = []

# Generators
train_dir = './train'
eval_dir = './eval'
training_set = TrajDataset(annotations_file='train_labels.csv', data_dir=train_dir, transform=normalize)
training_generator = torch.utils.data.DataLoader(training_set, **params)

validation_set = TrajDataset(annotations_file='eval_labels.csv', data_dir=eval_dir, transform=normalize)
validation_generator = torch.utils.data.DataLoader(validation_set, **params)

print('''
    Starting training:
        Epochs: {}
        Batch size: {}
        Learning rate: {}
        Training size: {}
        Validation size: {}
        Checkpoints: {}
        CUDA: {}
    '''.format(epochs, batch_size, lr, len(training_set),
               len(validation_set), str(save_cp), str(use_cuda)))

N_train = len(training_set)

optimizer = torch.optim.SGD(net.parameters(),
                          lr=lr,
                          momentum=0.9,
                          weight_decay=0.0005)

criterion = torch.nn.CrossEntropyLoss()

net = net.to(device)
for epoch in range(epochs):
    print('Starting epoch {}/{}.'.format(epoch + 1, epochs))
    net.train()
    #Training
    epoch_loss = 0
    i = 0
    for local_batch, local_labels in training_generator:
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        type_pred = net(local_batch.double())
        #pred_labels = type_pred.view(-1)
        #print(type_pred.shape)
        #true_label = local_labels.argmax()
        #print(true_labels_flat.shape)
        loss = criterion(type_pred, local_labels)
        epoch_loss += loss.item()

        print('{0:.4f} --- loss: {1:.6f}'.format(i * batch_size / N_train, loss.item()))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        
    print('Epoch finished ! Loss: {}'.format(epoch_loss / i))
    with torch.set_grad_enabled(False):
        #Validation
        val_score = eval_net(net, validation_generator, use_cuda)
        val_scores.append(val_score)
        print('Validation Score: {}'.format(val_score))

    if save_cp and (epoch % 10 == 0):
        torch.save(net.state_dict(),
                    dir_checkpoint + 'CP{}_conv.pth'.format(epoch + 1))
        print('Checkpoint {} saved !'.format(epoch + 1))
        
val_file = open('scores_conv.txt', 'w')
for i in range(len(val_scores)):
    val_file.write('{:.2f} '.format(val_scores[i]))
val_file.close()


    Starting training:
        Epochs: 20
        Batch size: 1
        Learning rate: 0.001
        Training size: 10
        Validation size: 4
        Checkpoints: 1
        CUDA: True
    
Starting epoch 1/20.
0.0000 --- loss: 1.153912
0.1000 --- loss: 1.046188
0.2000 --- loss: 1.103985
0.3000 --- loss: 1.105013
0.4000 --- loss: 1.151778
0.5000 --- loss: 1.103250
0.6000 --- loss: 1.038108
0.7000 --- loss: 1.149402
0.8000 --- loss: 1.044998
0.9000 --- loss: 1.041341
Epoch finished ! Loss: 1.0937974501055654
Validation Score: 0.25
Checkpoint 1 saved !
Starting epoch 2/20.
0.0000 --- loss: 1.141034
0.1000 --- loss: 1.142310
0.2000 --- loss: 1.138213
0.3000 --- loss: 1.034453
0.4000 --- loss: 1.101885
0.5000 --- loss: 1.100153
0.6000 --- loss: 1.033297
0.7000 --- loss: 1.034260
0.8000 --- loss: 1.096755
0.9000 --- loss: 1.042999
Epoch finished ! Loss: 1.0865358860788459
Validation Score: 0.25
Starting epoch 3/20.
0.0000 --- loss: 1.022276
0.1000 --- loss: 1.023120
0.2000 --- loss: 1.1

In [9]:
print(net)

conv_model(
  (conv_tr_block): Sequential(
    (0): ConvTranspose2d(801, 15, kernel_size=(2, 2), stride=(1, 1))
    (1): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ConvTranspose2d(15, 15, kernel_size=(2, 2), stride=(1, 1))
    (5): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): ConvTranspose2d(15, 15, kernel_size=(2, 2), stride=(1, 1))
    (9): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
  )
  (conv_block): Sequential(
    (0): Conv2d(15, 5, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU(inplace=True)
  )
  (flatten): Flatten()
  (linear_block): Sequential(
    (0): Linear(in_features=12500, out_features=256, bias